# Quantum Convolutional Neural Network with scikit-qulacs

[Quantum Convolutional Neural Networks](https://arxiv.org/abs/1810.03787)をTensorFlow Quantum（以降TFQ）で実装したチュートリアルの[Quantum Convolutional Neural Network](https://www.tensorflow.org/quantum/tutorials/qcnn?hl=en)（以降QCNN）のscikit-qulacs実装例です。

## インポート
`scikit-qulacs`の`QNNClassifier`と`create_qcnn_ansatz`をインポートします。`numpy`や`matplotlib.pyplot`等も併せてインポートします。

In [40]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
from sklearn.metrics import f1_score
from skqulacs.qnn import QNNClassifier
from skqulacs.circuit.pre_defined import create_qcnn_ansatz


## テストデータの生成
TFQ版QCNNと同様に、テストデータを生成します。
TFQ版はデータの中に量子状態を表す回路を埋め込みテンソルに変換して持たせます。
`scikit-qulacs`の場合、入力データはInputParameterとして`create_qcnn_ansatz`の中でエンコードされます。
そのため、データとしては単純に回転角と正解ラベルを持たせます。

In [41]:
def generate_data(bits: int, random_seed: int = 0):
    """Generate training and testing data."""
    rng = default_rng(random_seed)
    n_rounds = 20
    excitations = []
    labels = []
    for n in range(n_rounds):
        for bit in range(bits):
            r = rng.uniform(-np.pi, np.pi)
            excitations.append(r)
            labels.append(1 if (-np.pi / 2) <= r <= (np.pi / 2) else 0)

    split_ind = int(len(excitations) * 0.7)
    train_excitations = excitations[:split_ind]
    test_excitations = excitations[split_ind:]

    train_labels = labels[:split_ind]
    test_labels = labels[split_ind:]

    return train_excitations, np.array(train_labels), \
        test_excitations, np.array(test_labels)


## QCNN回路の作成
`create_qcnn_ansatz()`を呼び出して回路を作成してください。  
第一引数に量子ビットを指定します。現在は固定の8ビットに対応しています。  
第二引数は乱数のシード値を指定してください。

In [42]:
nqubit = 8        # 量子ビット数。現在8固定
random_seed = 0   # 乱数のシード値
circuit = create_qcnn_ansatz(nqubit, random_seed)

## QNNClassifierクラスの作成
作成した回路を`QNNClassifierクラス`に指定してください。  
第一引数に回路を指定します。第二引数に分類数を指定します。ここでは二値問題のため2を指定してください。  
第三引数に探索アルゴリズムを指定します。`Adam`を指定してください。

In [43]:
num_class = 2  # 分類数（ここでは2つに分類）
solver="Adam"  # 探索アルゴリズム。"Adam"を指定してください。
qcl = QNNClassifier(circuit, num_class, solver)

## 実行
`generate_data()`を呼び出してテストデータを生成し、`qcl.fit()`を実行し学習を行います。  
`opt_params`に学習されたパラメータが入ります。  
`qcl.predict()`で推論を行います。  
`sklearn.metrics`の`f1_score`を使用して結果の精度を計算します。

In [44]:
maxiter = 20  # ループの最大。これが多いほど、正確になるが、時間がかかる。
x_train, y_train, x_test, y_test = generate_data(nqubit)
opt_loss, opt_params = qcl.fit(x_train, y_train, maxiter)
print("trained parameters: ", opt_params)
y_pred = qcl.predict(x_test)
print("f1_score: ", f1_score(y_test, y_pred, average="weighted"))

trained parameters:  [ 0.86055566 -1.47257325 -2.84277113 -3.64540393  1.63813567  2.0645404
  2.81517295  1.47490504 -3.12438615  2.0480168  -3.09803583  1.29147571
 -2.03791584  2.14666105  0.29468943 -1.45328126 -0.48317123 -3.19778986
 -2.13290226  1.18254825  3.12406216  3.02117754  1.16579467  1.00568327
  1.20771264 -0.61939065 -2.29275628  1.35106086  0.15930591 -1.19228546
 -0.08899907  2.44722424  2.72717584 -0.89349913 -1.01843381 -0.6809779
  2.45216607 -1.7143194   0.77400766 -2.61370868  2.09006482  1.80389187
 -1.63759009  2.36552019 -2.77359884 -1.02970688 -2.19735892 -0.31202696
 -0.59971848 -1.89429841 -2.57137445  0.50474327 -1.2648295   1.08067569
 -1.88800015  2.7778786  -0.98279722 -2.28221508  1.03800729  2.64329979
 -0.31567124  2.91988853  3.29547352  3.01294356 -0.25104379  1.61935809
 -0.01619368  0.18417373  1.79564452 -0.53623311  1.47330373  1.32664983
  2.71471107 -2.41944962  1.43894442  2.68558375  3.02343761  2.87273628
 -2.2068808   2.96961442  2.4500

0.9以上の正解率となります。